In [ ]:
import math
import itertools
import time
from collections import defaultdict

# Top 10 stations data
stations = [
    {"name": "Stagg St & Union Ave", "demand_hour": -11.583261, "lat": 40.708745, "lng": -73.951036},
    {"name": "Old Slip & South St", "demand_hour": -10.305316, "lat": 40.703393, "lng": -74.007804},
    {"name": "W 45 St & 8 Ave", "demand_hour": -11.008410, "lat": 40.759470, "lng": -73.989216},
    {"name": "8 Ave & W 31 St", "demand_hour": -10.164487, "lat": 40.750814, "lng": -73.994662},
    {"name": "W 52 St & 11 Ave", "demand_hour": -9.818765, "lat": 40.767257, "lng": -73.994027},
    {"name": "E 3 St & Ave A", "demand_hour": -8.518913, "lat": 40.723537, "lng": -73.985112},
    {"name": "Schermerhorn St & Court St", "demand_hour": -6.293553, "lat": 40.691169, "lng": -73.992027},
    {"name": "Greenwich Ave & Charles St", "demand_hour": -6.761417, "lat": 40.734961, "lng": -74.000262},
    {"name": "Grand Army Plaza & Plaza St West", "station_id": 4010.15, "demand_hour": -8.849465, "lat": 40.672968, "lng": -73.970880},
    {"name": "E 68 St & Madison Ave", "station_id": 6932.15, "demand_hour": -5.318388, "lat": 40.769081, "lng": -73.967165}
]

# Calculate demands as absolute values of negative demand_hour, rounded up to whole bikes
for station in stations:
    station["demand"] = math.ceil(abs(station["demand_hour"]))

# Configuration
NUM_TRUCKS = 4
TRUCK_CAPACITY = 30
print(f"Finding optimal routes for {len(stations)} stations with {NUM_TRUCKS} trucks (capacity: {TRUCK_CAPACITY} bikes each)")

# Define depot location (average of stations)
depot_lat = sum(station["lat"] for station in stations) / len(stations)
depot_lng = sum(station["lng"] for station in stations) / len(stations)
print(f"Depot location: Lat={depot_lat}, Lng={depot_lng}")

# All locations (depot + stations)
locations = [{"name": "Depot", "lat": depot_lat, "lng": depot_lng, "demand": 0}] + stations

def haversine_distance(lat1, lng1, lat2, lng2):
    """Calculate distance between two geographic coordinates in kilometers"""
    R = 6371  # Earth radius in kilometers
    
    # Convert to radians
    lat1, lng1, lat2, lng2 = map(math.radians, [lat1, lng1, lat2, lng2])
    
    dlat = lat2 - lat1
    dlng = lng2 - lng1
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlng/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    return R * c

# Calculate distance matrix
print("Calculating distance matrix...")
dist_matrix = []
for i, loc1 in enumerate(locations):
    row = []
    for j, loc2 in enumerate(locations):
        if i == j:
            row.append(0)
        else:
            # Distance in kilometers
            distance = haversine_distance(loc1["lat"], loc1["lng"], loc2["lat"], loc2["lng"])
            row.append(distance)
    dist_matrix.append(row)

# Print distance matrix (just first few rows for brevity)
print("\nDistance Matrix (first 3 rows, kilometers):")
for i in range(min(3, len(dist_matrix))):
    formatted_row = [f"{val:.2f}" for val in dist_matrix[i]]
    print(f"{i}: {formatted_row}")

# Print demand for each station
print("\nStation demands (number of bikes needed):")
total_demand = 0
for i, station in enumerate(stations, 1):
    print(f"{i}: {station['name']}: {station['demand']} bikes")
    total_demand += station['demand']
print(f"Total demand: {total_demand} bikes")
print(f"Total capacity: {NUM_TRUCKS * TRUCK_CAPACITY} bikes")

if total_demand > NUM_TRUCKS * TRUCK_CAPACITY:
    print("WARNING: Total demand exceeds total capacity!")

# We need to partition the stations among the trucks
# For small problems, we can enumerate all possible partitioning
def generate_partitions(stations, num_trucks):
    """Generate all possible ways to partition stations among trucks, respecting capacity constraints"""
    station_indices = list(range(1, len(stations) + 1))  # 1-indexed for stations
    
    # Track the number of valid partitions
    valid_partitions = 0
    
    # Function to check if a partition respects capacity constraints
    def is_valid_partition(partition):
        for group in partition:
            total_demand = sum(stations[i-1]["demand"] for i in group)
            if total_demand > TRUCK_CAPACITY:
                return False
        return True
    
    # Generate all possible partitions using set partitioning
    for partition in bell_number_partitions(station_indices, num_trucks):
        # Only consider partitions with exactly num_trucks groups (some might be empty)
        if len(partition) == num_trucks:
            if is_valid_partition(partition):
                valid_partitions += 1
                yield partition
    
    print(f"Total valid partitions: {valid_partitions}")

def bell_number_partitions(elements, max_groups):
    """Generate all possible partitions of elements into at most max_groups groups"""
    if not elements:
        yield []
        return
    
    element = elements[0]
    for partition in bell_number_partitions(elements[1:], max_groups):
        # Add to existing group
        for i, group in enumerate(partition):
            new_partition = partition.copy()
            new_partition[i] = group + [element]
            if len(new_partition) <= max_groups:
                yield new_partition
        
        # Create new group if not exceeding max_groups
        if len(partition) < max_groups:
            yield partition + [[element]]

# Calculate optimal route for a single partition
def calculate_optimal_routes(partition):
    """Calculate the optimal route for each group in the partition"""
    total_distance = 0
    routes = []
    
    for group in partition:
        # Skip empty groups
        if not group:
            routes.append(([0, 0], 0))  # Empty route (depot to depot)
            continue
        
        # Find the optimal route for this group
        best_distance = float('inf')
        best_route = None
        
        # Check all permutations of the stations in this group
        for perm in itertools.permutations(group):
            # Complete route starts and ends at depot
            route = [0] + list(perm) + [0]
            
            # Calculate total distance
            route_distance = 0
            for i in range(len(route) - 1):
                from_node = route[i]
                to_node = route[i + 1]
                route_distance += dist_matrix[from_node][to_node]
            
            if route_distance < best_distance:
                best_distance = route_distance
                best_route = route
        
        routes.append((best_route, best_distance))
        total_distance += best_distance
    
    return routes, total_distance

# Find the optimal solution
print("\nGenerating and evaluating all valid partitions...")
start_time = time.time()

best_partition = None
best_routes = None
best_total_distance = float('inf')
partition_count = 0

for partition in generate_partitions(stations, NUM_TRUCKS):
    partition_count += 1
    
    # For progress tracking
    if partition_count % 100 == 0:
        elapsed = time.time() - start_time
        print(f"Evaluated {partition_count} partitions in {elapsed:.2f} seconds")
    
    routes, total_distance = calculate_optimal_routes(partition)
    
    if total_distance < best_total_distance:
        best_total_distance = total_distance
        best_partition = partition
        best_routes = routes

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nSearch complete in {elapsed_time:.2f} seconds")
print(f"Evaluated {partition_count:,} different partitions")

if best_partition:
    print("\nOptimal solution found:")
    print(f"Total distance: {best_total_distance:.2f} km")
    
    print("\nStation assignment to trucks:")
    for i, (group, (route, distance)) in enumerate(zip(best_partition, best_routes)):
        group_demand = sum(stations[j-1]["demand"] for j in group)
        print(f"Truck {i+1} - Stations: {group}, Total demand: {group_demand}/{TRUCK_CAPACITY}, Distance: {distance:.2f} km")
    
    print("\nDetailed routes:")
    for i, (route, distance) in enumerate(best_routes):
        print(f"\nTruck {i+1} Route (Total: {distance:.2f} km):")
        if len(route) <= 2:  # Just depot to depot (empty route)
            print("  No stations assigned (empty route)")
            continue
            
        for j in range(len(route)):
            node = route[j]
            if node == 0:
                if j == 0:
                    print(f"  Start at Depot")
                else:
                    print(f"  Return to Depot")
            else:
                station = locations[node]
                print(f"  Visit {station['name']} (Need: {station['demand']} bikes)")
        
        print("\n  Distance breakdown:")
        for j in range(len(route) - 1):
            from_node = route[j]
            to_node = route[j + 1]
            from_name = locations[from_node]['name']
            to_name = locations[to_node]['name']
            seg_distance = dist_matrix[from_node][to_node]
            print(f"    {from_name} to {to_name}: {seg_distance:.2f} km")
else:
    print("No valid solution found that satisfies all constraints.")

Finding optimal routes for 10 stations with 4 trucks (capacity: 30 bikes each)
Depot location: Lat=40.7281395, Lng=-73.9852191
Calculating distance matrix...

Distance Matrix (first 3 rows, kilometers):
0: ['0.00', '3.60', '3.35', '3.50', '2.64', '4.41', '0.51', '4.15', '1.48', '6.25', '4.80']
1: ['3.60', '0.00', '4.82', '6.49', '5.95', '7.45', '3.31', '3.97', '5.07', '4.32', '6.85']
2: ['3.35', '4.82', '0.00', '6.43', '5.39', '7.20', '2.95', '1.90', '3.57', '4.60', '8.07']

Station demands (number of bikes needed):
1: Stagg St & Union Ave: 12 bikes
2: Old Slip & South St: 11 bikes
3: W 45 St & 8 Ave: 12 bikes
4: 8 Ave & W 31 St: 11 bikes
5: W 52 St & 11 Ave: 10 bikes
6: E 3 St & Ave A: 9 bikes
7: Schermerhorn St & Court St: 7 bikes
8: Greenwich Ave & Charles St: 7 bikes
9: Grand Army Plaza & Plaza St West: 9 bikes
10: E 68 St & Madison Ave: 6 bikes
Total demand: 94 bikes
Total capacity: 120 bikes

Generating and evaluating all valid partitions...
Evaluated 100 partitions in 0.00 secon

In [7]:
import pulp
import math
import time

# Top 10 stations data
stations = [
    {"name": "Stagg St & Union Ave", "demand_hour": -11.583261, "lat": 40.708745, "lng": -73.951036},
    {"name": "Old Slip & South St", "demand_hour": -10.305316, "lat": 40.703393, "lng": -74.007804},
    {"name": "W 45 St & 8 Ave", "demand_hour": -11.008410, "lat": 40.759470, "lng": -73.989216},
    {"name": "8 Ave & W 31 St", "demand_hour": -10.164487, "lat": 40.750814, "lng": -73.994662},
    {"name": "W 52 St & 11 Ave", "demand_hour": -9.818765, "lat": 40.767257, "lng": -73.994027},
    {"name": "E 3 St & Ave A", "demand_hour": -8.518913, "lat": 40.723537, "lng": -73.985112},
    {"name": "Schermerhorn St & Court St", "demand_hour": -6.293553, "lat": 40.691169, "lng": -73.992027},
    {"name": "Greenwich Ave & Charles St", "demand_hour": -6.761417, "lat": 40.734961, "lng": -74.000262},
    {"name": "Grand Army Plaza & Plaza St West", "demand_hour": -8.849465, "lat": 40.672968, "lng": -73.970880},
    {"name": "E 68 St & Madison Ave", "demand_hour": -5.318388, "lat": 40.769081, "lng": -73.967165}
]

# Configuration
NUM_TRUCKS = 4
TRUCK_CAPACITY = 30

# Convert negative demand to positive (number of bikes needed)
for station in stations:
    station["demand"] = math.ceil(abs(station["demand_hour"]))

# Define depot location (average of stations)
depot_lat = sum(station["lat"] for station in stations) / len(stations)
depot_lng = sum(station["lng"] for station in stations) / len(stations)

print(f"Depot location: Lat={depot_lat}, Lng={depot_lng}")

# Create a list of all locations (depot + stations)
locations = [{"name": "Depot", "lat": depot_lat, "lng": depot_lng, "demand": 0}] + stations

# Calculate distance matrix between all locations
def haversine_distance(lat1, lng1, lat2, lng2):
    """Calculate distance between two geographic coordinates in kilometers"""
    R = 6371  # Earth radius in kilometers
    
    # Convert to radians
    lat1, lng1, lat2, lng2 = map(math.radians, [lat1, lng1, lat2, lng2])
    
    dlat = lat2 - lat1
    dlng = lng2 - lng1
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlng/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    return R * c

# Create the distance matrix
dist_matrix = []
for i, loc1 in enumerate(locations):
    row = []
    for j, loc2 in enumerate(locations):
        if i == j:
            row.append(0)
        else:
            # Distance in kilometers
            distance = haversine_distance(loc1["lat"], loc1["lng"], loc2["lat"], loc2["lng"])
            row.append(distance)
    dist_matrix.append(row)

print("\nDistance Matrix (first few rows, kilometers):")
for i in range(min(3, len(dist_matrix))):
    formatted_row = [f"{val:.2f}" for val in dist_matrix[i]]
    print(f"{i}: {formatted_row}")

# Print demand for each station
print("\nStation demands (number of bikes needed):")
total_demand = 0
for i, station in enumerate(stations, 1):
    print(f"{i}: {station['name']}: {station['demand']} bikes")
    total_demand += station['demand']
print(f"Total demand: {total_demand} bikes")
print(f"Total capacity: {NUM_TRUCKS * TRUCK_CAPACITY} bikes")

if total_demand > NUM_TRUCKS * TRUCK_CAPACITY:
    print("WARNING: Total demand exceeds total capacity!")

print("\nSolving CVRP using PuLP with MILP approach...")
start_time = time.time()

# Create the problem
prob = pulp.LpProblem("Bike_Sharing_CVRP", pulp.LpMinimize)

# Sets
num_locations = len(locations)
depot = 0  # Depot index
stations_indices = range(1, num_locations)  # Stations (excluding depot)
all_indices = range(num_locations)  # All locations including depot
trucks = range(NUM_TRUCKS)  # Trucks

# Decision variables:
# x[i,j,k] = 1 if truck k travels from location i to location j
x = {}
for k in trucks:
    for i in all_indices:
        for j in all_indices:
            if i != j:  # No self-loops
                x[i, j, k] = pulp.LpVariable(f'x_{i}_{j}_{k}', cat='Binary')

# Objective: minimize total distance
prob += pulp.lpSum(dist_matrix[i][j] * x[i, j, k] for k in trucks for i in all_indices for j in all_indices if i != j)

# Constraint: Each station must be visited exactly once
for j in stations_indices:
    prob += pulp.lpSum(x[i, j, k] for i in all_indices for k in trucks if i != j) == 1

# Constraint: Each truck leaves the depot at most once
for k in trucks:
    prob += pulp.lpSum(x[depot, j, k] for j in stations_indices) <= 1

# Constraint: Flow conservation - each truck that enters a node must leave it
for k in trucks:
    for h in all_indices:
        prob += pulp.lpSum(x[i, h, k] for i in all_indices if i != h) == pulp.lpSum(x[h, j, k] for j in all_indices if j != h)

# Constraint: Capacity constraint for each truck
for k in trucks:
    prob += pulp.lpSum(stations[j-1]["demand"] * x[i, j, k] for i in all_indices for j in stations_indices if i != j) <= TRUCK_CAPACITY

# Constraint: Subtour elimination using MTZ formulation
u = {}
for k in trucks:
    for i in stations_indices:
        u[i, k] = pulp.LpVariable(f'u_{i}_{k}', lowBound=2, upBound=num_locations, cat='Integer')

for k in trucks:
    for i in stations_indices:
        for j in stations_indices:
            if i != j:
                prob += u[i, k] - u[j, k] + 1 <= (num_locations - 1) * (1 - x[i, j, k])

# Solve the problem with a time limit
solver = pulp.PULP_CBC_CMD(timeLimit=300)  # 5-minute time limit
prob.solve(solver)

end_time = time.time()
elapsed_time = end_time - start_time

# Print solution status
print(f"\nSolution status: {pulp.LpStatus[prob.status]}")
print(f"Solving time: {elapsed_time:.2f} seconds")

if prob.status == pulp.LpStatusOptimal or prob.status == pulp.LpStatusNotSolved:
    # Calculate objective value (total distance)
    objective_value = pulp.value(prob.objective)
    print(f"Total distance: {objective_value:.2f} km")
    
    # Extract and print routes
    print("\nRoutes:")
    for k in trucks:
        # Check if truck is used
        if any(pulp.value(x[depot, j, k]) > 0.5 for j in stations_indices):
            # Start at depot
            route = [depot]
            current = depot
            
            # Find the next station until we return to depot
            while True:
                for j in all_indices:
                    if j != current and pulp.value(x[current, j, k]) > 0.5:
                        route.append(j)
                        current = j
                        break
                if current == depot:
                    break
            
            # Calculate the total load for this route
            load = sum(stations[j-1]["demand"] for j in route if j != depot)
            
            # Calculate route distance
            route_distance = sum(dist_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
            
            # Print the route
            route_names = [locations[i]["name"] for i in route]
            print(f"Truck {k+1}: {' -> '.join(route_names)}")
            print(f"  Load: {load}/{TRUCK_CAPACITY} bikes")
            print(f"  Distance: {route_distance:.2f} km")
        else:
            print(f"Truck {k+1}: Not used")
else:
    print("No optimal solution found. The problem may be infeasible or unbounded.")

# Print mathematical formulation explanation
print("\nMathematical Formulation Used:")
print("Mixed-Integer Linear Programming (MILP) model with:")
print("- Binary variables x[i,j,k] for truck k traveling from i to j")
print("- Integer variables u[i,k] for Miller-Tucker-Zemlin subtour elimination")
print("- Objective: Minimize total distance traveled")
print("- Constraints: Flow conservation, capacity limits, and subtour elimination")
print("- Solution method: Branch and Cut (via CBC solver)")

Depot location: Lat=40.7281395, Lng=-73.9852191

Distance Matrix (first few rows, kilometers):
0: ['0.00', '3.60', '3.35', '3.50', '2.64', '4.41', '0.51', '4.15', '1.48', '6.25', '4.80']
1: ['3.60', '0.00', '4.82', '6.49', '5.95', '7.45', '3.31', '3.97', '5.07', '4.32', '6.85']
2: ['3.35', '4.82', '0.00', '6.43', '5.39', '7.20', '2.95', '1.90', '3.57', '4.60', '8.07']

Station demands (number of bikes needed):
1: Stagg St & Union Ave: 12 bikes
2: Old Slip & South St: 11 bikes
3: W 45 St & 8 Ave: 12 bikes
4: 8 Ave & W 31 St: 11 bikes
5: W 52 St & 11 Ave: 10 bikes
6: E 3 St & Ave A: 9 bikes
7: Schermerhorn St & Court St: 7 bikes
8: Greenwich Ave & Charles St: 7 bikes
9: Grand Army Plaza & Plaza St West: 9 bikes
10: E 68 St & Madison Ave: 6 bikes
Total demand: 94 bikes
Total capacity: 120 bikes

Solving CVRP using PuLP with MILP approach...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/tianzeyin/cs5800/venv/lib/python3.13/site-packages/pu

In [ ]:
import pandas as pd
import numpy as np
import math
import time
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Load the forecast data
print("Loading forecast data...")
forecasts = pd.read_csv('citibike_station_demand_forecasts.csv')
print(f"Loaded {len(forecasts)} stations")

# Filter stations with negative demand_hour (expected outflow)
outflow_stations = forecasts[forecasts['demand_hour'] < 0].copy()

# Sort by demand_hour (most negative first)
outflow_stations = outflow_stations.sort_values(by='demand_hour')

# Print the first 10 stations with highest outflow
print("\nTop 10 stations with highest expected outflow:")
print(outflow_stations.head(10)[['station_name', 'station_id', 'demand_hour', 'demand_day', 'lat', 'lng']])

# Calculate total demand (outflow) for next hour
total_outflow = outflow_stations['demand_hour'].sum()
total_outflow_absolute = abs(total_outflow)
print(f"\nTotal expected outflow in the next hour: {total_outflow_absolute:.2f} bikes")

# Calculate count of stations needing bikes
outflow_count = len(outflow_stations)
print(f"Number of stations with expected outflow: {outflow_count}")

# Configuration
NUM_TRUCKS = 66
TRUCK_CAPACITY = 30

# Convert negative demand to positive (number of bikes needed)
outflow_stations['demand'] = outflow_stations['demand_hour'].apply(lambda x: math.ceil(abs(x)))

# Check if total demand can be met
total_demand = outflow_stations['demand'].sum()
total_capacity = NUM_TRUCKS * TRUCK_CAPACITY
print(f"\nTotal demand: {total_demand} bikes")
print(f"Total capacity: {total_capacity} bikes")

if total_demand > total_capacity:
    print(f"WARNING: Total demand exceeds total capacity! Need {total_demand - total_capacity} more bikes capacity.")
    # Adjust by focusing on stations with highest needs
    outflow_stations = outflow_stations.sort_values(by='demand', ascending=False).head(NUM_TRUCKS * TRUCK_CAPACITY // 3)
    total_demand = outflow_stations['demand'].sum()
    print(f"Adjusted to focus on {len(outflow_stations)} highest-need stations with total demand: {total_demand} bikes")

# Define depot location (average of all stations, which is a common approach)
depot_lat = outflow_stations['lat'].mean()
depot_lng = outflow_stations['lng'].mean()
print(f"\nDepot location: Lat={depot_lat}, Lng={depot_lng}")

# Set up data for OR-Tools
# 0 will be the depot, 1...N will be the stations
locations = [(depot_lat, depot_lng)] + list(zip(outflow_stations['lat'], outflow_stations['lng']))
demands = [0] + list(outflow_stations['demand'])  # Depot has no demand
station_names = ["Depot"] + list(outflow_stations['station_name'])

# Calculate distance matrix
def haversine_distance(lat1, lng1, lat2, lng2):
    """Calculate distance between two geographic coordinates in meters"""
    R = 6371000  # Earth radius in meters
    
    # Convert to radians
    lat1, lng1, lat2, lng2 = map(math.radians, [lat1, lng1, lat2, lng2])
    
    dlat = lat2 - lat1
    dlng = lng2 - lng1
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlng/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    return R * c

print("\nCalculating distance matrix...")
start_time = time.time()
dist_matrix = []
for i, loc1 in enumerate(locations):
    row = []
    for j, loc2 in enumerate(locations):
        if i == j:
            row.append(0)
        else:
            # Distance in meters
            distance = int(haversine_distance(loc1[0], loc1[1], loc2[0], loc2[1]))
            row.append(distance)
    dist_matrix.append(row)
print(f"Distance matrix calculation took {time.time() - start_time:.2f} seconds")

# Create the routing index manager
manager = pywrapcp.RoutingIndexManager(len(locations), NUM_TRUCKS, 0)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Define distance callback
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Capacity constraint
def demand_callback(from_index):
    """Returns the demand of the node."""
    from_node = manager.IndexToNode(from_index)
    return demands[from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    [TRUCK_CAPACITY] * NUM_TRUCKS,  # vehicle capacity
    True,  # start cumul to zero
    'Capacity')

# Setting first solution heuristic
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

# Set metaheuristic
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

# Set time limit
search_parameters.time_limit.seconds = 300  # 5 minutes

# Limit the number of solutions to find
search_parameters.solution_limit = 1

# Set first solution strategy
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem
print("\nSolving the routing problem using OR-Tools...")
start_time = time.time()
solution = routing.SolveWithParameters(search_parameters)
solve_time = time.time() - start_time

# Print solution
if solution:
    print(f"Solution found in {solve_time:.2f} seconds")
    
    # Get total distance
    total_distance = 0
    total_bikes_delivered = 0
    trucks_used = 0
    max_bikes_per_truck = 0
    
    routes_info = []
    
    for vehicle_id in range(NUM_TRUCKS):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route = []
        
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            route_load += demands[node_index]
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        
        route.append(manager.IndexToNode(index))  # Add depot at the end
        
        # Only count if truck is actually used (visits stations)
        if len(route) > 2:  # More than just depot->depot
            trucks_used += 1
            total_distance += route_distance
            total_bikes_delivered += route_load
            max_bikes_per_truck = max(max_bikes_per_truck, route_load)
            
            routes_info.append({
                'truck_id': vehicle_id + 1,
                'route': route,
                'stations_visited': len(route) - 2,  # Excluding depot at start and end
                'distance': route_distance,
                'load': route_load
            })
    
    # Print summary
    print(f"\nSolution summary:")
    print(f"Total distance: {total_distance/1000:.2f} km")
    print(f"Total bikes delivered: {total_bikes_delivered}")
    print(f"Trucks used: {trucks_used} out of {NUM_TRUCKS}")
    print(f"Maximum bikes per truck: {max_bikes_per_truck} out of capacity {TRUCK_CAPACITY}")
    print(f"Average bikes per truck: {total_bikes_delivered/trucks_used:.2f}")
    print(f"Average distance per truck: {total_distance/(1000*trucks_used):.2f} km")
    
    # Print a sample of routes (first 5)
    print("\nSample of truck routes:")
    for i, route_info in enumerate(sorted(routes_info, key=lambda x: -x['load'])[:5]):
        truck_id = route_info['truck_id']
        route = route_info['route']
        stations = route_info['stations_visited']
        distance = route_info['distance']
        load = route_info['load']
        
        print(f"Truck {truck_id}: {stations} stations, {load} bikes, {distance/1000:.2f} km")
        print("  Route: Depot -> ", end="")
        for node in route[1:-1]:
            print(f"{station_names[node]} ({demands[node]} bikes) -> ", end="")
        print("Depot")
        print()
    
    # Print info about OR-Tools solution method
    print("\nOR-Tools Algorithm Information:")
    print(f"First solution strategy: PATH_CHEAPEST_ARC")
    print(f"Metaheuristic: GUIDED_LOCAL_SEARCH")
    
    # Determine what method was used (without trying to access possibly missing attributes)
    if routing.status() == 1:  # ROUTING_SUCCESS
        print("Solution method: Complete optimization (found optimal solution)")
    elif solution:
        print("Solution method: Heuristic optimization (found feasible solution)")
        
    # More detailed stats - fixed to avoid errors
    print(f"Solution status: {routing.status()}")
    
    # Try to access statistics safely (no method will cause error)
    try:
        solver = routing.solver()
        print(f"Solver wall time: {solver.WallTime()/1000:.2f} seconds")
    except Exception as e:
        print("Note: Some solver statistics are not available in this version of OR-Tools")
    
else:
    print("No solution found!")

Loading forecast data...
Loaded 2188 stations

Top 10 stations with highest expected outflow:
                  station_name station_id  demand_hour  demand_day        lat  \
3        Madison Ave & E 99 St    7443.01    -1.727555  -48.598507  40.789808   
7              108 St & 39 Ave    6490.02    -1.305722  -34.362849  40.751676   
8            57 St & Grand Ave    5440.05    -1.839180  -44.655856  40.719181   
9       Melrose Ave & E 154 St    7918.12    -0.371512   -6.574438  40.819567   
10     Wyckoff Ave & Gates Ave    4847.03    -0.417540   -7.156026  40.699911   
11            12 Ave & W 40 St    6765.01    -0.576258  -28.328146  40.760965   
12         31 St & Newtown Ave     6923.2    -2.000000  -48.000000  40.767382   
14  St Nicholas Ave & W 137 St    7898.03    -1.582244  -40.841331  40.818465   
15     Clinton St & Tillary St    4748.07    -0.368277    8.913621  40.696233   
16             5 Av & W 139 St    7851.04    -0.530773  -16.233122  40.815484   

          lng 